In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch08Joins").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_15773


In [5]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")

In [6]:
person.show(5, False)

+---+----------------+----------------+---------------+
|id |name            |graduate_program|spark_status   |
+---+----------------+----------------+---------------+
|0  |Bill Chambers   |0               |[100]          |
|1  |Matei Zaharia   |1               |[500, 250, 100]|
|2  |Michael Armbrust|1               |[250, 100]     |
+---+----------------+----------------+---------------+



In [7]:
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")

In [8]:
graduateProgram.show(5,False)

+---+-------+---------------------+-----------+
|id |degree |department           |school     |
+---+-------+---------------------+-----------+
|0  |Masters|School of Information|UC Berkeley|
|2  |Masters|EECS                 |UC Berkeley|
|1  |Ph.D.  |EECS                 |UC Berkeley|
+---+-------+---------------------+-----------+



In [9]:
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [10]:
sparkStatus.show(5, False)

+---+--------------+
|id |status        |
+---+--------------+
|500|Vice President|
|250|PMC Member    |
|100|Contributor   |
+---+--------------+



In [11]:
# COMMAND ----------
joinExpression = person["graduate_program"] == graduateProgram['id']

In [12]:
# COMMAND ----------
wrongJoinExpression = person["name"] == graduateProgram["school"]

In [24]:
# COMMAND ----------
joinType = "inner"
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [22]:
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression).show(8, False)
#Deafault Join is Inner Join

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [25]:
joinType='outer'
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
|null    |null            |null            |null           |2  |Masters|EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [26]:
joinType='outer'
graduateProgram.join(person.withColumnRenamed("id", "personId"), joinExpression, joinType).show(8, False)

+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|id |degree |department           |school     |personId|name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0       |Bill Chambers   |0               |[100]          |
|1  |Ph.D.  |EECS                 |UC Berkeley|1       |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Ph.D.  |EECS                 |UC Berkeley|2       |Michael Armbrust|1               |[250, 100]     |
|2  |Masters|EECS                 |UC Berkeley|null    |null            |null            |null           |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+



In [27]:
joinType='left_outer'
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [28]:
joinType='left_outer'
graduateProgram.join(person.withColumnRenamed("id", "personId"), joinExpression, joinType).show(8, False)

+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|id |degree |department           |school     |personId|name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0       |Bill Chambers   |0               |[100]          |
|1  |Ph.D.  |EECS                 |UC Berkeley|1       |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Ph.D.  |EECS                 |UC Berkeley|2       |Michael Armbrust|1               |[250, 100]     |
|2  |Masters|EECS                 |UC Berkeley|null    |null            |null            |null           |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+



In [29]:
joinType='right_outer'
person.withColumnRenamed("id", "personId").join(graduateProgram, joinExpression, joinType).show(8, False)

+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|personId|name            |graduate_program|spark_status   |id |degree |department           |school     |
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+
|0       |Bill Chambers   |0               |[100]          |0  |Masters|School of Information|UC Berkeley|
|1       |Matei Zaharia   |1               |[500, 250, 100]|1  |Ph.D.  |EECS                 |UC Berkeley|
|2       |Michael Armbrust|1               |[250, 100]     |1  |Ph.D.  |EECS                 |UC Berkeley|
|null    |null            |null            |null           |2  |Masters|EECS                 |UC Berkeley|
+--------+----------------+----------------+---------------+---+-------+---------------------+-----------+



In [30]:
joinType='right_outer'
graduateProgram.join(person.withColumnRenamed("id", "personId"), joinExpression, joinType).show(8, False)

+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|id |degree |department           |school     |personId|name            |graduate_program|spark_status   |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+
|0  |Masters|School of Information|UC Berkeley|0       |Bill Chambers   |0               |[100]          |
|1  |Ph.D.  |EECS                 |UC Berkeley|1       |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Ph.D.  |EECS                 |UC Berkeley|2       |Michael Armbrust|1               |[250, 100]     |
+---+-------+---------------------+-----------+--------+----------------+----------------+---------------+

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 37334)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib64/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python2.7/SocketServer.py", line 649, in __init__
    self.handle()
  File "/usr/hdp/current/spark2-client/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/hdp/current/spark2-client/python/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError


In [17]:
# COMMAND ----------
gradProgram2 = graduateProgram.union(
    spark.createDataFrame([(0, "Masters", "Duplicated Row", "Duplicated School")]))

In [18]:
gradProgram2.show(5, False)

+---+-------+---------------------+-----------------+
|id |degree |department           |school           |
+---+-------+---------------------+-----------------+
|0  |Masters|School of Information|UC Berkeley      |
|2  |Masters|EECS                 |UC Berkeley      |
|1  |Ph.D.  |EECS                 |UC Berkeley      |
|0  |Masters|Duplicated Row       |Duplicated School|
+---+-------+---------------------+-----------------+



In [19]:
# COMMAND ----------
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId")\
  .join(sparkStatus, expr("array_contains(spark_status, id)")).show(10, False)

+--------+----------------+----------------+---------------+---+--------------+
|personId|name            |graduate_program|spark_status   |id |status        |
+--------+----------------+----------------+---------------+---+--------------+
|0       |Bill Chambers   |0               |[100]          |100|Contributor   |
|1       |Matei Zaharia   |1               |[500, 250, 100]|500|Vice President|
|1       |Matei Zaharia   |1               |[500, 250, 100]|250|PMC Member    |
|1       |Matei Zaharia   |1               |[500, 250, 100]|100|Contributor   |
|2       |Michael Armbrust|1               |[250, 100]     |250|PMC Member    |
|2       |Michael Armbrust|1               |[250, 100]     |100|Contributor   |
+--------+----------------+----------------+---------------+---+--------------+

